In [1]:
import os
import re

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash
px.defaults.template = "simple_white"
pio.templates.default = "plotly_white"
os.getcwd()

'/home/viet/OneDrive/Studying_Materials/Data_Visualization/FootballVizualization/oss'

In [30]:
file_name_dict = {
    "zone_against" : "stage-attempt-zones-against.csv",
    "zone_for" : "stage-attempt-zones-for.csv",
    "direction_against" : "stage-attempt-directions-against.csv",
    "direction_for" : "stage-attempt-directions-for.csv",
    "offensive" : "stage-team-stats-offensive.csv"
}
ATTEMPT = ["attempt-directions", "attempt-zones"]
GOAL_PASS = ["goals", "passes"]
FOR_AGAINST = [*ATTEMPT, *GOAL_PASS]
ALL_TYPE = [*FOR_AGAINST, "cards", "team", "touch"]
LEAGUE = ["Bundesliga", "EPL", "LaLiga", "Ligue1", "SerieA"]


def _add_suffix_columns(df, suffix):
    col_name_to_change = list(df.columns)
    col_name_to_change.remove("Team")

    df.rename(
            columns={
                col_name:f"{col_name}_{suffix}" for col_name in col_name_to_change
            },
            inplace=True
        )


def gather_all_league():
    df = pd.DataFrame()
    for league in LEAGUE:
        df = pd.concat([df, gather_all_seasons(league)], ignore_index=True)
    return df


def gather_one_season(league, season):
    first = True
    df_league = pd.DataFrame()
    season_path = os.path.join("../data", league)

    for type in FOR_AGAINST: # everything csv file that has for and against
        for mode in ["for", "against"]:
            df_temp = pd.read_csv(os.path.join(season_path, f"{season}/stage-{type}-{mode}.csv"), dtype=str)
            df_temp.drop(["R"], axis=1, inplace=True)
            df_temp.rename(
                columns = {
                    'Left Side':'left',
                    'Attempts from the middle':'middle',
                    'Right Side': 'right',
                }, 
            inplace = True
    )
            if first:
                df_league = df_temp
                first = False
            else:
                df_league = df_league.merge(df_temp, on="Team", suffixes=("_for","_against"))

    type = "cards" # handle cards file
    df_temp = pd.read_csv(os.path.join(season_path, f"{season}/stage-{type}.csv"), dtype=str)
    df_temp.drop(["R"], axis=1, inplace=True)
    df_league = df_league.merge(df_temp, on="Team")

    type = "team" # handle team files
    for mode in ["defensive", "offensive", "detailed", "summary"]:
        df_temp = pd.read_csv(os.path.join(season_path, f"{season}/stage-{type}-stats-{mode}.csv"), dtype=str)

        # format Team column in this fucker table
        df_temp["Team"] = df_temp["Team"].apply(lambda s: re.sub(re.compile(r"\d{1,2}\.\s"), "", s))

        _add_suffix_columns(df_temp, mode)

        df_league = df_league.merge(df_temp, on="Team")
    
    type = "touch" # handle touch files
    for mode in ["channels", "zones"]:
        df_temp = pd.read_csv(os.path.join(season_path, f"{season}/stage-{type}-{mode}.csv"), dtype=str)
        df_temp.drop(["R"], axis=1, inplace=True)
        if mode == "channels":
            _add_suffix_columns(df_temp, mode)
        
        df_league = df_league.merge(df_temp, on="Team")

    df_league = df_league.applymap(lambda x: int(x[:-1])/100 if x.__contains__("%") else x)

    season = season.split("_")[0]
    df_league["season"] = [int(season) for i in range(len(df_league))]
    df_league["league"] = [league for i in range(len(df_league))]

    df_league.rename(
            columns = {
                'Team':'team', 
                'Rating_summary':'rating'
            }, 
            inplace = True
    )
    df_league["rating"] = pd.to_numeric(df_league["rating"])
    df_league["Shots pg_offensive"] = pd.to_numeric(df_league["Shots pg_offensive"])
    df_league["Shots pg_defensive"] = pd.to_numeric(df_league["Shots pg_defensive"])
    df_league["Shots OT pg_offensive"] = pd.to_numeric(df_league["Shots OT pg_offensive"])
    df_league["Dribbles pg_offensive"] = pd.to_numeric(df_league["Dribbles pg_offensive"])
    df_league["Fouled pg_offensive"] = pd.to_numeric(df_league["Fouled pg_offensive"])

    df_league.drop(columns=['Rating_offensive', 'Rating_defensive', 'Rating_detailed'], inplace=True)
    return df_league


def gather_all_seasons(league):
    df = pd.DataFrame()

    season_path = os.path.join("../data", league)
    for season in os.listdir(season_path): # iterate through each season
        df = pd.concat([df, gather_one_season(league, season)], ignore_index=True)

    return df.sort_values(by=['season'])

In [31]:
test = gather_one_season("SerieA", "2009_2010")
print(list(test.columns))
test.tail()

['team', 'left_for', 'middle_for', 'right_for', 'left_against', 'middle_against', 'right_against', 'In 6 Yards Box_for', 'In 18 Yards Box_for', 'Outside of Box_for', 'In 6 Yards Box_against', 'In 18 Yards Box_against', 'Outside of Box_against', 'Open Play_for', 'Counter Attack_for', 'Set Piece_for', 'Penalty_for', 'Own Goal_for', 'Open Play_against', 'Counter Attack_against', 'Set Piece_against', 'Penalty_against', 'Own Goal_against', 'Cross pg_for', 'Through Ball pg_for', 'Long Balls pg_for', 'Short Passes pg_for', 'Cross pg_against', 'Through Ball pg_against', 'Long Balls pg_against', 'Short Passes pg_against', 'Fouls', 'Unprofessional', 'Dive', 'Other', 'Shots pg_defensive', 'Tackles pg_defensive', 'Interceptions pg_defensive', 'Fouls pg_defensive', 'Offsides pg_defensive', 'Shots pg_offensive', 'Shots OT pg_offensive', 'Dribbles pg_offensive', 'Fouled pg_offensive', 'Total_detailed', 'OutOfBox_detailed', 'SixYardBox_detailed', 'PenaltyArea_detailed', 'Goals_summary', 'Shots pg_summ

,team,left_for,middle_for,right_for,left_against,middle_against,right_against,In 6 Yards Box_for,In 18 Yards Box_for,Outside of Box_for,...,AerialsWon_summary,rating,Left Side_channels,Middle of the pitch_channels,Right Side_channels,Own Third,Middle Third,Opposition Third,season,league
15,Napoli,0.18,0.62,0.21,0.19,0.67,0.15,0.06,0.48,0.46,...,8.9,6.96,0.29,0.32,0.40,0.25,0.46,0.30,2009,SerieA
16,Catania,0.16,0.70,0.15,0.24,0.60,0.16,0.04,0.43,0.53,...,9.5,6.84,0.34,0.33,0.33,0.27,0.47,0.26,2009,SerieA
17,Bologna,0.15,0.65,0.20,0.18,0.66,0.16,0.05,0.42,0.53,...,9.1,6.78,0.34,0.33,0.32,0.30,0.45,0.26,2009,SerieA
18,Livorno,0.15,0.60,0.25,0.21,0.62,0.17,0.05,0.35,0.60,...,8,6.70,0.32,0.31,0.38,0.28,0.45,0.26,2009,SerieA
19,Chievo,0.13,0.73,0.14,0.21,0.60,0.19,0.10,0.48,0.42,...,10.4,6.84,0.28,0.33,0.38,0.27,0.45,0.28,2009,SerieA


In [32]:
df = gather_all_league()

In [33]:
df.head()

,team,left_for,middle_for,right_for,left_against,middle_against,right_against,In 6 Yards Box_for,In 18 Yards Box_for,Outside of Box_for,...,AerialsWon_summary,rating,Left Side_channels,Middle of the pitch_channels,Right Side_channels,Own Third,Middle Third,Opposition Third,season,league
0,Nuernberg,0.27,0.52,0.22,0.23,0.62,0.15,0.05,0.48,0.47,...,5.8,6.74,0.37,0.26,0.37,0.28,0.45,0.27,2009,Bundesliga
1,Hamburger SV,0.27,0.55,0.18,0.21,0.60,0.19,0.04,0.51,0.45,...,5.7,6.88,0.40,0.24,0.36,0.27,0.47,0.27,2009,Bundesliga
2,Wolfsburg,0.26,0.59,0.15,0.21,0.58,0.21,0.08,0.58,0.34,...,5.6,6.95,0.37,0.26,0.37,0.28,0.44,0.28,2009,Bundesliga
3,Werder Bremen,0.25,0.60,0.15,0.24,0.55,0.21,0.08,0.53,0.39,...,7.2,7.06,0.39,0.26,0.34,0.24,0.45,0.31,2009,Bundesliga
4,Borussia M.Gladbach,0.25,0.58,0.17,0.21,0.62,0.17,0.04,0.47,0.49,...,7.6,6.74,0.37,0.27,0.36,0.29,0.46,0.24,2009,Bundesliga


In [6]:
len(df)

1176

### Average percentage of attempts in each ZONE for each league

#### In 6 Yards Box

##### For

In [22]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["In 6 Yards Box_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team In 6 Yards Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [23]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["In 6 Yards Box_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team In 6 Yards Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

#### In 18 Yards Box

##### For

In [24]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["In 18 Yards Box_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team In 18 Yards Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [25]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["In 18 Yards Box_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team In 18 Yards Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

#### Outside of Box

##### For

In [26]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["Outside of Box_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team Outside of Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [27]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["Outside of Box_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team Outside of Box",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

### Average percentage of attempts in each DIRECTION for each league

#### Left

##### For

In [7]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["left_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team from the left",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [8]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["left_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team from the left",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

#### Right

##### For

In [9]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["right_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team from the right",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [10]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["right_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team from the right",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

#### Middle

##### For

In [11]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["middle_for"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts for each team from the middle",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

##### Against

In [12]:
fig = go.Figure()
for league in LEAGUE:
    temp = df[df["league"] == league].groupby("season")["middle_against"].mean()
    fig.add_scatter(x=temp.index, y=temp.values, name=league, mode="lines")
fig.update_layout(
    title="Average percentage of attempts against each team from the middle",
    yaxis=dict(
        tickformat=".0%",
        showgrid=True
    ),
    xaxis=dict(
        tickvals=[i for i in range(2009, 2022)],
        ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                    "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        showgrid=True
    ),
    xaxis_title="Season",
    yaxis_title="Attempts (%)"
)
fig.show()

### Interactive percentage of attempts for the last 10 years of each team

#### With team filter

##### From each direction

In [13]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H3('League'),
            dcc.Checklist(
                options=[
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/germany-flag-icon-16.png"),
                                " Bundesliga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Bundesliga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/england-flag-icon-16.png", alt='image'),
                                " EPL",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "EPL",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/spain-flag-icon-16.png", alt='image'),
                                " LaLiga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "LaLiga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/france-flag-icon-16.png", alt='image'),
                                " Ligue 1",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Ligue1",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/italy-flag-icon-16.png", alt='image'),
                                " Serie A",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "SerieA",
                    },
                ],
                value=["Bundesliga"],
                id="league",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Direction'),
            dcc.RadioItems(
                {
                    "right": "Right Side",
                    "left": "Left Side",
                    "middle": "Middle"
                },
                "right",
                id="direction",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Attempts for or against'),
            dcc.RadioItems(
                {
                    "for": "Attempts for",
                    "against": "Attempts against"
                },
                "for",
                id="for_or_against",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Team'),
            dcc.Dropdown([], id='team_dropdown', multi=True),
            html.Div(id='dd-output-container')
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    
    dcc.Graph(
        id='graph',
    )
])


@app.callback(
    Output("team_dropdown", "options"),
    Input('league', 'value'))
def update_team_dropdown(selected_league):
    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])
    if mask is not None:
        filtered_df = df[mask]
        return sorted(filtered_df["team"].unique())
    return []


@app.callback(
    Output('graph', 'figure'),
    Input('team_dropdown', 'value'),
    Input('league', 'value'),
    Input('direction', 'value'),
    Input('for_or_against', 'value'))
def update_figure(selected_team, selected_league, selected_dir, selected_for_or_against):
    if len(selected_league) == 0:  # if no league is selected then return an empty plot
        fig = px.line()

        fig.update_layout(
            yaxis=dict(
                tickformat=".0%",
                showgrid=True
            ),
            xaxis=dict(
                tickvals=[i for i in range(2009, 2022)],
                ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                          "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
                showgrid=True
            ),
            showlegend=False
        )
        return fig

    # filtered_df = df[df["for_or_against"]
    #                           == selected_for_or_against]

    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])
    filtered_df_league = df[mask]

    mask_team = None
    if ctx.triggered_id == "team_dropdown":
        for i in range(len(selected_team)):
            if i == 0:
                mask_team = (df.team == selected_team[i])
            else:
                mask_team |= (df.team == selected_team[i])

    if mask_team is not None: 
        mask_team &= mask
        filtered_df_league_team = df[mask_team]

        league_str = ", ".join(selected_league)
        fig = px.line(filtered_df_league_team, x="season", y=f"{selected_dir}_{selected_for_or_against}", color="team",
                title=f"<b>Percentage of attempts from the {selected_dir} in {league_str} league</b>", markers=True,
                hover_name="team", hover_data=["season", f"{selected_dir}_{selected_for_or_against}", "league"])

        fig.update_layout(
            title_font_size=20,
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
            )
        )

        fig.update_traces(
            opacity=0.4,
        )
    else:
        league_str = ", ".join(selected_league)
        fig = px.line(filtered_df_league, x="season", y=f"{selected_dir}_{selected_for_or_against}", color="team",
                    title=f"<b>Percentage of attempts from the {selected_dir} in {league_str} league</b>", markers=True,
                    hover_name="team", hover_data=["season", f"{selected_dir}_{selected_for_or_against}", "league"])

        fig.update_layout(
            title_font_size=20,
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
            ),
            showlegend=False
        )

        fig.update_traces(
            opacity=0.4,
            line_color='rgb(189,189,189)'
        )
    mean = filtered_df_league.groupby("season")[f"{selected_dir}_{selected_for_or_against}"].mean()
    fig.add_trace(go.Scatter(
        x=mean.index,
        y=mean.values,
        line=dict(
            width=5,
            color="rgb(49,130,189)"
        ),
        name="Average",
        showlegend=False
    ))

    fig.update_layout(
        yaxis=dict(
            tickformat=".0%",
            showgrid=True
        ),
        xaxis=dict(
            tickvals=[i for i in range(2009, 2022)],
            ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                      "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
            showgrid=True
        ),
        xaxis_title="Season",
        yaxis_title="Attempts (%)"
    )

    return fig

app.run_server(debug=True, mode="external")

Dash app running on http://127.0.0.1:8050/


##### In each zone

In [14]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H3('League'),
            dcc.Checklist(
                options=[
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/germany-flag-icon-16.png"),
                                " Bundesliga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Bundesliga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/england-flag-icon-16.png", alt='image'),
                                " EPL",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "EPL",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/spain-flag-icon-16.png", alt='image'),
                                " LaLiga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "LaLiga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/france-flag-icon-16.png", alt='image'),
                                " Ligue 1",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Ligue1",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/italy-flag-icon-16.png", alt='image'),
                                " Serie A",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "SerieA",
                    },
                ],
                value=["Bundesliga"],
                id="league",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Zone'),
            dcc.RadioItems(
                {
                    "In 6 Yards Box": "In 6 Yards Box",
                    "In 18 Yards Box": "In 18 Yards Box",
                    "Outside of Box": "Outside of Box"
                },
                "In 6 Yards Box",
                id="zone",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Attempts for or against'),
            dcc.RadioItems(
                {
                    "for": "Attempts for",
                    "against": "Attempts against"
                },
                "for",
                id="for_or_against",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Team'),
            dcc.Dropdown([], id='team_dropdown', multi=True),
            html.Div(id='dd-output-container')
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    
    dcc.Graph(
        id='graph',
    )
])


@app.callback(
    Output("team_dropdown", "options"),
    Input('league', 'value'))
def update_team_dropdown(selected_league):
    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])
    if mask is not None:
        filtered_df = df[mask]
        return sorted(filtered_df["team"].unique())
    return []


@app.callback(
    Output('graph', 'figure'),
    Input('team_dropdown', 'value'),
    Input('league', 'value'),
    Input('zone', 'value'),
    Input('for_or_against', 'value'))
def update_figure(selected_team, selected_league, selected_zone, selected_for_or_against):
    if len(selected_league) == 0:  # if no league is selected then return an empty plot
        fig = px.line()

        fig.update_layout(
            yaxis=dict(
                tickformat=".0%",
                showgrid=True
            ),
            xaxis=dict(
                tickvals=[i for i in range(2009, 2022)],
                ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                          "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
                showgrid=True
            ),
            showlegend=False
        )
        return fig

    # filtered_df = df[df["for_or_against"]
    #                           == selected_for_or_against]

    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])
    filtered_df_league = df[mask]

    mask_team = None
    if ctx.triggered_id == "team_dropdown":
        for i in range(len(selected_team)):
            if i == 0:
                mask_team = (df.team == selected_team[i])
            else:
                mask_team |= (df.team == selected_team[i])

    if mask_team is not None: 
        mask_team &= mask
        filtered_df_league_team = df[mask_team]

        league_str = ", ".join(selected_league)
        fig = px.line(filtered_df_league_team, x="season", y=f"{selected_zone}_{selected_for_or_against}", color="team",
                title=f"<b>Percentage of attempts from the {selected_zone} in {league_str} league</b>", markers=True,
                hover_name="team", hover_data=["season", f"{selected_zone}_{selected_for_or_against}", "league"])

        fig.update_layout(
            title_font_size=20,
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
            )
        )

        fig.update_traces(
            opacity=0.4,
        )
    else:
        league_str = ", ".join(selected_league)
        fig = px.line(filtered_df_league, x="season", y=f"{selected_zone}_{selected_for_or_against}", color="team",
                    title=f"<b>Percentage of attempts from the {selected_zone} in {league_str} league</b>", markers=True,
                    hover_name="team", hover_data=["season", f"{selected_zone}_{selected_for_or_against}", "league"])

        fig.update_layout(
            title_font_size=20,
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
            ),
            showlegend=False
        )

        fig.update_traces(
            opacity=0.4,
            line_color='rgb(189,189,189)'
        )
    mean = filtered_df_league.groupby("season")[f"{selected_zone}_{selected_for_or_against}"].mean()
    fig.add_trace(go.Scatter(
        x=mean.index,
        y=mean.values,
        line=dict(
            width=5,
            color="rgb(49,130,189)"
        ),
        name="Average",
        showlegend=False
    ))

    fig.update_layout(
        yaxis=dict(
            tickformat=".0%",
            showgrid=True
        ),
        xaxis=dict(
            tickvals=[i for i in range(2009, 2022)],
            ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                      "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
            showgrid=True
        ),
        xaxis_title="Season",
        yaxis_title="Attempts (%)"
    )

    return fig

app.run_server(debug=True, mode="external")

Dash app running on http://127.0.0.1:8050/


#### With top k best performing teams in each season

##### For each direction

In [15]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H3('League'),
            dcc.Checklist(
                options=[
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/germany-flag-icon-16.png"),
                                " Bundesliga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Bundesliga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/england-flag-icon-16.png", alt='image'),
                                " EPL",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "EPL",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/spain-flag-icon-16.png", alt='image'),
                                " LaLiga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "LaLiga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/france-flag-icon-16.png", alt='image'),
                                " Ligue 1",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Ligue1",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/italy-flag-icon-16.png", alt='image'),
                                " Serie A",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "SerieA",
                    },
                ],
                value=["Bundesliga"],
                id="league",
                labelStyle={'display': 'block'},
                style={"height": 150, "width": 200, "overflow": "auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Direction'),
            dcc.RadioItems(
                {
                    "right": "Right Side",
                    "left": "Left Side",
                    "middle": "Middle"
                },
                "right",
                id="direction",
                labelStyle={'display': 'block'},
                style={"height": 150, "width": 200, "overflow": "auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Attempts for or against'),
            dcc.RadioItems(
                {
                    "for": "Attempts for",
                    "against": "Attempts against"
                },
                "for",
                id="for_or_against",
                labelStyle={'display': 'block'},
                style={"height": 150, "width": 200, "overflow": "auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Highlight top performing teams in each season'),
            dcc.Input(id='top', type='number', min=0, max=30, step=1, value=0)
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    dcc.Graph(
        id='graph',
    )
])


@app.callback(
    Output('graph', 'figure'),
    Input('top', 'value'),
    Input('league', 'value'),
    Input('direction', 'value'),
    Input('for_or_against', 'value'))
def update_figure(selected_top, selected_league, selected_dir, selected_for_or_against):
    if len(selected_league) == 0:  # if no league is selected then return an empty plot
        fig = px.line()

        fig.update_layout(
            yaxis=dict(
                tickformat=".0%",
                showgrid=True
            ),
            xaxis=dict(
                tickvals=[i for i in range(2009, 2022)],
                ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                          "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
                showgrid=True
            ),
            showlegend=False
        )
        return fig

    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])

    filtered_df_league = df[mask]

    league_str = ", ".join(selected_league)
    fig = px.line(filtered_df_league, x="season", y=f"{selected_dir}_{selected_for_or_against}", color="team",
                  title=f"<b>Percentage of attempts from the {selected_dir} in {league_str} league</b>", markers=True,
                  hover_name="team", hover_data=["season", f"{selected_dir}_{selected_for_or_against}", "league"])

    fig.update_layout(
        title_font_size=20,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
        ),
        showlegend=False
    )

    fig.update_traces(
        opacity=0.4,
        line_color='rgb(189,189,189)'
    )

    # add highlight dots
    top_k_teams = filtered_df_league.set_index(["team", f"{selected_dir}_{selected_for_or_against}"]).groupby(
        "season")["rating"].nlargest(selected_top).reset_index()
    fig.add_scatter(x=top_k_teams["season"], y=top_k_teams[f"{selected_dir}_{selected_for_or_against}"],
                    name="", marker_color="orange", mode="markers", marker_size=12, marker_symbol='star', marker_line_color="black", marker_line_width=1)

    mean = filtered_df_league.groupby(
        "season")[f"{selected_dir}_{selected_for_or_against}"].mean()
    fig.add_trace(go.Scatter(
        x=mean.index,
        y=mean.values,
        line=dict(
            width=5,
            color="rgb(49,130,189)"
        ),
        name="Average",
        showlegend=False
    ))

    fig.update_layout(
        yaxis=dict(
            tickformat=".0%",
            showgrid=True
        ),
        xaxis=dict(
            tickvals=[i for i in range(2009, 2022)],
            ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                      "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
            showgrid=True
        ),
        xaxis_title="Season",
        yaxis_title="Attempts (%)"
    )

    return fig


app.run_server(debug=True, mode="external")


Dash app running on http://127.0.0.1:8050/


##### For each zone

In [16]:
from dash import dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H3('League'),
            dcc.Checklist(
                options=[
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/germany-flag-icon-16.png"),
                                " Bundesliga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Bundesliga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/england-flag-icon-16.png", alt='image'),
                                " EPL",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "EPL",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/spain-flag-icon-16.png", alt='image'),
                                " LaLiga",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "LaLiga",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/france-flag-icon-16.png", alt='image'),
                                " Ligue 1",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "Ligue1",
                    },
                    {
                        "label": html.Div(
                            [
                                html.Img(
                                    src="./assets/flags/italy-flag-icon-16.png", alt='image'),
                                " Serie A",
                            ], style={'display': 'inline-block', 'marginTop': '5px'}
                        ),
                        "value": "SerieA",
                    },
                ],
                value=["Bundesliga"],
                id="league",
                labelStyle={'display': 'block'},
                style={"height": 150, "width": 200, "overflow": "auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Zone'),
            dcc.RadioItems(
                {
                    "In 6 Yards Box": "In 6 Yards Box",
                    "In 18 Yards Box": "In 18 Yards Box",
                    "Outside of Box": "Outside of Box"
                },
                "In 6 Yards Box",
                id="zone",
                labelStyle={'display': 'block'},
                style={"height":150, "width":200, "overflow":"auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Attempts for or against'),
            dcc.RadioItems(
                {
                    "for": "Attempts for",
                    "against": "Attempts against"
                },
                "for",
                id="for_or_against",
                labelStyle={'display': 'block'},
                style={"height": 150, "width": 200, "overflow": "auto"}
            )
        ], style={'padding': 10, 'flex': 1}),
        html.Div([
            html.H3('Highlight top performing teams in each season'),
            dcc.Input(id='top', type='number', min=0, max=30, step=1, value=0)
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    dcc.Graph(
        id='graph',
    )
])


@app.callback(
    Output('graph', 'figure'),
    Input('top', 'value'),
    Input('league', 'value'),
    Input('zone', 'value'),
    Input('for_or_against', 'value'))
def update_figure(selected_top, selected_league, selected_zone, selected_for_or_against):
    if len(selected_league) == 0:  # if no league is selected then return an empty plot
        fig = px.line()

        fig.update_layout(
            yaxis=dict(
                tickformat=".0%",
                showgrid=True
            ),
            xaxis=dict(
                tickvals=[i for i in range(2009, 2022)],
                ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                          "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
                showgrid=True
            ),
            showlegend=False
        )
        return fig

    mask = None
    for i in range(len(selected_league)):
        if i == 0:
            mask = (df.league == selected_league[i])
        else:
            mask |= (df.league == selected_league[i])

    filtered_df_league = df[mask]

    league_str = ", ".join(selected_league)
    fig = px.line(filtered_df_league, x="season", y=f"{selected_zone}_{selected_for_or_against}", color="team",
                  title=f"<b>Percentage of attempts from the {selected_zone} in {league_str} league</b>", markers=True,
                  hover_name="team", hover_data=["season", f"{selected_zone}_{selected_for_or_against}", "league"])

    fig.update_layout(
        title_font_size=20,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
        ),
        showlegend=False
    )

    fig.update_traces(
        opacity=0.4,
        line_color='rgb(189,189,189)'
    )

    # add highlight dots
    top_k_teams = filtered_df_league.set_index(["team", f"{selected_zone}_{selected_for_or_against}"]).groupby(
        "season")["rating"].nlargest(selected_top).reset_index()
    fig.add_scatter(x=top_k_teams["season"], y=top_k_teams[f"{selected_zone}_{selected_for_or_against}"],
                    name="", marker_color="orange", mode="markers", marker_size=12, marker_symbol='star', marker_line_color="black", marker_line_width=1)
                    
    mean = filtered_df_league.groupby(
        "season")[f"{selected_zone}_{selected_for_or_against}"].mean()
    fig.add_trace(go.Scatter(
        x=mean.index,
        y=mean.values,
        line=dict(
            width=5,
            color="rgb(49,130,189)"
        ),
        name="Average",
        showlegend=False
    ))

    fig.update_layout(
        yaxis=dict(
            tickformat=".0%",
            showgrid=True
        ),
        xaxis=dict(
            tickvals=[i for i in range(2009, 2022)],
            ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                      "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
            showgrid=True
        ),
        xaxis_title="Season",
        yaxis_title="Attempts (%)"
    )

    return fig


app.run_server(debug=True, mode="external")


Dash app running on http://127.0.0.1:8050/


In [17]:
selected_league = ["EPL"]
mask = None
for i in range(len(selected_league)):
    if i == 0:
        mask = (df.league == selected_league[i])
    else:
        mask |= (df.league == selected_league[i])

filtered_df_league = df[mask]

In [18]:
filtered_df_league.head()

,team,left_for,middle_for,right_for,left_against,middle_against,right_against,In 6 Yards Box_for,In 18 Yards Box_for,Outside of Box_for,...,AerialsWon_summary,rating,Left Side_channels,Middle of the pitch_channels,Right Side_channels,Own Third,Middle Third,Opposition Third,season,league
216,Blackburn,0.21,0.67,0.12,0.17,0.68,0.14,0.11,0.51,0.38,...,21.2,6.80,0.38,0.32,0.30,0.30,0.39,0.31,2009,EPL
217,Aston Villa,0.21,0.66,0.12,0.14,0.72,0.15,0.10,0.51,0.39,...,18.8,7.06,0.40,0.26,0.35,0.30,0.42,0.29,2009,EPL
218,Wigan,0.20,0.60,0.20,0.14,0.69,0.17,0.07,0.38,0.55,...,14.9,6.81,0.34,0.29,0.37,0.29,0.43,0.27,2009,EPL
219,Chelsea,0.19,0.64,0.17,0.22,0.61,0.17,0.08,0.47,0.45,...,11.6,7.30,0.36,0.33,0.31,0.25,0.44,0.32,2009,EPL
220,Birmingham,0.19,0.63,0.18,0.16,0.66,0.17,0.09,0.47,0.44,...,16.9,6.91,0.36,0.26,0.38,0.31,0.40,0.29,2009,EPL


In [19]:
filtered_df_league.set_index("team").groupby("season")["rating"].nlargest(3)[2009]

team
Chelsea              7.30
Manchester United    7.19
Tottenham            7.19
Name: rating, dtype: float64

In [20]:
filtered_df_league.set_index(["team", "left_for"]).groupby("season")["rating"].nlargest(0).reset_index()

,season,team,left_for,rating


In [21]:
selected_dir = "left"
selected_for_or_against = "for"
selected_top = 4
fig = go.Figure()
top_k_teams = filtered_df_league.set_index(["team", f"{selected_dir}_{selected_for_or_against}"]).groupby("season")["rating"].nlargest(selected_top).reset_index()
fig.add_scatter(x=top_k_teams["season"], y=top_k_teams[f"{selected_dir}_{selected_for_or_against}"],
                name="bruh", marker_color="orange", mode="markers", marker_size=12)

### Shots target per game

In [35]:
df["Shots not OT pg_offensive"] = df["Shots pg_offensive"] - df["Shots OT pg_offensive"]
df[["Shots not OT pg_offensive", "Shots OT pg_offensive", "Shots pg_offensive"]].head()

,Shots not OT pg_offensive,Shots OT pg_offensive,Shots pg_offensive
0,8.6,3.9,12.5
1,8.7,5.6,14.3
2,8.9,5.4,14.3
3,9.3,6.3,15.6
4,8.5,4.7,13.2


In [110]:
def stacked_bar_shots(league):
    temp = df[df["league"] == league].groupby("season")[["Shots OT pg_offensive", "Shots not OT pg_offensive", "Shots pg_offensive"]].mean()
    fig = go.Figure(
        data=[
            go.Bar(
                name="Shots on target",
                x=temp.index,
                y=temp["Shots OT pg_offensive"],
                marker_color="indianred",
                text=temp["Shots OT pg_offensive"].round(2),
                textposition='auto',
            ),
            go.Bar(
                name="Shots missed target",
                x=temp.index,
                y=temp["Shots not OT pg_offensive"],
                marker_color="aquamarine",
                text=temp["Shots pg_offensive"].round(2),
                textposition='auto',
            )
        ]
    )
    fig.update_layout(
        barmode='stack',
        bargap=0.3,
        title=f"Shots per game in each season in {league}",
        xaxis=dict(
            tickvals=[i for i in range(2009, 2021)],
            ticktext=["2009-2010", "2010-2011", "2011-2012", "2012-2013", "2013-2014", "2014-2015",
                        "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022"],
        ),
        xaxis_title="Season",
        yaxis_title="Shots",
        uniformtext_minsize=5, 
        uniformtext_mode='hide'
    )
    fig.show()

In [111]:
stacked_bar_shots(LEAGUE[0])

In [100]:
stacked_bar_shots(LEAGUE[1])

In [101]:
stacked_bar_shots(LEAGUE[2])

In [102]:
stacked_bar_shots(LEAGUE[3])

In [103]:
stacked_bar_shots(LEAGUE[4])

#### Shot distribution

In [120]:
def box_plot_shot(league, shot_col):
    temp = df[df["league"] == league]
    fig = go.Figure()
    for season in range(2009, 2021):
        fig.add_trace(
            go.Box(
                y=temp[shot_col],
                name=season, 
                fillcolor='yellow',
                line={'color' : 'orange'},
                showlegend=False
            )
        )
    fig.show()

In [121]:
box_plot_shot(LEAGUE[0], "Shots pg_offensive")

In [122]:
box_plot_shot(LEAGUE[0], "Shots OT pg_offensive")